In [1]:
import io
import numpy as np

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds

# 데이터 준비하기

In [2]:
def _normalize_img(img, label):
    img = tf.cast(img, tf.float32) / 255.
    return (img, label)

train_dataset, test_dataset = tfds.load(name="mnist", split=['train', 'test'], as_supervised=True)

# Build your input pipelines
train_dataset = train_dataset.shuffle(1024).batch(32)
train_dataset = train_dataset.map(_normalize_img)

test_dataset = test_dataset.batch(32)
test_dataset = test_dataset.map(_normalize_img)

# 모델 빌드하기

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28,28,1)),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=2),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation=None), # No activation on final dense layer
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1)) # L2 normalize embeddings

])

# 훈련 및 평가하기

In [4]:
# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [5]:
# Train the network
history = model.fit(
    train_dataset,
    epochs=5)

Epoch 1/5
1875/1875 [==============================] - 13s 5ms/step - loss: 0.5886
Epoch 2/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4513
Epoch 3/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4183
Epoch 4/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.4017
Epoch 5/5
1875/1875 [==============================] - 10s 5ms/step - loss: 0.3917


In [6]:
# Evaluate the network
results = model.predict(test_dataset)

In [7]:
# Save test embeddings for visualization in projector
np.savetxt("vecs.tsv", results, delimiter='\t')

out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for img, labels in tfds.as_numpy(test_dataset):
    [out_m.write(str(x) + "\n") for x in labels]
out_m.close()


In [8]:
try:
  from google.colab import files
  files.download('vecs.tsv')
  files.download('meta.tsv')
except:
  pass

In [9]:
train_dataset

<MapDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [10]:
test_dataset

<MapDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>